## Model 1.0
Powered by [Eleonora Priori](https://www.est-en.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo) and [Pietro Terna](https://terna.to.it/) 


In [1]:
%%javascript
// to avoid scroll in windows
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

====================================================================================================

## 1

import libs \
MPI init \
context and runner definition \
t(), T(), Tc() function definitions \
random number generator rng creation \
initialization of the parameters from yaml file\
memory allocations to manage ghosts


====================================================================================================

In [2]:
import time
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from math import ceil
from typing import Tuple, List, Dict
import numpy as np
import csv
import os
import sys


comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)

# tick number
def t():
    return int(runner.schedule.tick)

#Initializes the repast4py.parameters.params dictionary with the model input parameters.
params = parameters.init_params("model1.yaml", "")

#moves to the right folder (that you must create and initialize with a firm-features.csv file)
if not os.path.isdir(params["log_file_root"]):
    print("There is no "+params["log_file_root"] + " starting folder!")  
    sys.exit(0)
else: os.chdir(params["log_file_root"])


#generate random seed
repast4py.random.init(rng_seed=params['myRandom.seed'][rank]) #each rank has a seed
rng = repast4py.random.default_rng 


#timer T()
startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime
T() #launches the timer

#cpuTimer Tc()
startCpuTime=-1
def Tc():
    global startCpuTime
    if startCpuTime < 0:
        startCpuTime=time.process_time()
    return time.process_time() - startCpuTime
Tc() #launches the cpu timer

agent_cache={} # dict with uid as keys and agents' tuples as values

===================================================================================================

## 2

create agents' classes and restore_agent function 



===================================================================================================

In [3]:
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, labor:int, capital:float,\
                 duration:int, recipe: float, laborProductivity: float, maxOrderProduction: float,\
                 assetsUsefulLife: float, markup: float, orderObservationFrequency: int, \
                 dimensionalClass: int):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid
        self.labor=labor
        self.capital=capital
        self.availableLabor=self.labor
        self.availableCapital=self.capital
        self.duration=duration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.maxOrderProduction=maxOrderProduction
        self.assetsUsefulLife=assetsUsefulLife
        self.markup=markup
        self.orderObservationFrequency=orderObservationFrequency
        self.dimensionalClass=dimensionalClass
        
        self.lostProduction=0
        self.inventories=0
        self.inProgressInventories=0
        self.appRepository=[] #aPP is aProductiveProcess
        
        self.profits=0
        self.revenues=0
        self.totalCosts=0
        self.totalCostOfLabor=0
        self.totalCostOfCapital=0
        self.addedValue=0
        self.finalInventories=0
        self.initialInventories=0
        self.myBalancesheet=np.zeros((params['howManyCycles'], 13))
        
        self.movAvElements=[]
        self.movAvElementNum=0
        
    def dealingMovAvElements(self, freq, x):
        
        self.movAvElements.append(x)
        if len(self.movAvElements) > freq: self.movAvElements.pop(0) 

        
    def receivingNewOrder(self, productionOrder:float):
        
        #creates a statistics of the values of the received order
        self.dealingMovAvElements(self.orderObservationFrequency, productionOrder)
        
        #decision on accepting or refusing the new order
        productionOrderQuantityByPeriod=productionOrder/self.duration
        requiredLabor=np.ceil(productionOrderQuantityByPeriod/self.laborProductivity)
        requiredCapital=requiredLabor*self.recipe
        
        #create a new aPP or skip the order
        if requiredLabor <= self.labor and requiredCapital <= self.capital: 
            aProductiveProcess = ProductiveProcess(productionOrderQuantityByPeriod, \
                                                   requiredLabor, requiredCapital, self.duration)
            self.appRepository.append(aProductiveProcess)
        
        

    def produce(self)->tuple: 
        
        #total values of the firm in the current interval unit
        self.currentTotalCostOfProductionOrder=0
        self.currentTotalOutput=0
        self.currentTotalCostOfUnusedFactors=0
        self.currentTotalLostProduction=0
        self.currentTotalCostOfLostProduction=0
        
        avgRequiredLabor=0
        avgRequiredCapital=0
        
        for aProductiveProcess in self.appRepository:  
            
            if not aProductiveProcess.hasResources and \
                        (self.availableLabor >= aProductiveProcess.requiredLabor and\
                         self.availableCapital >= aProductiveProcess.requiredCapital):
                self.availableLabor -= aProductiveProcess.requiredLabor
                self.availableCapital -= aProductiveProcess.requiredCapital
                aProductiveProcess.hasResources = True 
                
            if aProductiveProcess.hasResources: #resources may be just assigned above
                #production
                (aPPoutputOfThePeriod, aPPrequiredLabor, aPPrequiredCapital, aPPlostProduction, aPPcostOfLostProduction)=\
                aProductiveProcess.step()
                     
                self.currentTotalOutput += aPPoutputOfThePeriod
                
                cost = aPPrequiredLabor*params['wage'] + aPPrequiredCapital*params['costOfCapital']
                self.currentTotalCostOfProductionOrder += cost
                
                self.currentTotalLostProduction += aPPlostProduction
                self.currentTotalCostOfLostProduction += aPPcostOfLostProduction
              
        
                if aProductiveProcess.failure:
                    self.inProgressInventories -= cost*(aProductiveProcess.productionClock-1)
                    aProductiveProcess.productionClock=self.duration
                
                else:
                    if aProductiveProcess.productionClock < aProductiveProcess.duration:
                        self.inProgressInventories += cost
                    else:
                        self.inventories+=cost*self.duration
                        self.inProgressInventories -= cost*(self.duration-1)

        self.currentTotalCostOfUnusedFactors =  self.availableLabor*params['wage'] + \
                                        self.availableCapital*params['costOfCapital']
        
        self.totalCostOfLabor= self.labor*params['wage']
        self.totalCostOfCapital=self.capital*params['costOfCapital']
        
        
        if t() % self.orderObservationFrequency==0:
            #frequency of labor adjustments
            #print("ORDER MOV AV",self.uid, sum(self.movAvElements)/ len(self.movAvElements), flush=True)
            avgRequiredLabor=ceil(sum(self.movAvElements)/ len(self.movAvElements))
            avgRequiredCapital=avgRequiredLabor*self.recipe
            
            if self.labor > (1+params['tollerance']) * avgRequiredLabor:
                self.labor = int((1+params['tollerance']) * avgRequiredLabor) #max accepted q. of L (firing)
            if self.labor < (1/(1+params['tollerance'])) * avgRequiredLabor:
                self.labor = int((1/(1+params['tollerance'])) * avgRequiredLabor) #min accepted q. of L (hiring)
            #print(self.uid, "L", self.labor, flush=True)
       
        if self.capital >(1+params['tollerance']) * avgRequiredCapital: #no substitution of the prod. assets
                self.capital *= (1 - 1/(self.assetsUsefulLife * params['timeFraction']))              
        if self.capital <(1/(1+params['tollerance'])) * avgRequiredCapital: #increasing prod. assets
            self.capital = int((1/(1+params['tollerance'])) * avgRequiredCapital) 
        #print(self.uid, "K", self.capital, flush=True)            
                
            
        # remove concluded aPPs from the list
        for i in range(len(self.appRepository)-1,-1,-1):
            if self.appRepository[i].productionClock == self.appRepository[i].duration: 
                self.availableLabor+=self.appRepository[i].requiredLabor
                self.availableCapital+=self.appRepository[i].requiredCapital
                #self.appRepository.remove(aProductiveProcess)
                del self.appRepository[i]

        
        #print(self.uid,"\n", productionOrder,  self.labor, self.capital, self.recipe,\
              #self.laborProductivity,self.duration,"\n",\
              #output, self.currentTotalCostOfProductionOrder, self.currentTotalCostOfUnusedFactors, self.inventories,\
              #self.inProgressInventories, self.currentTotalLostProduction,\
              #self.currentTotalCostOfLostProduction, flush=True)
        
        return(self.currentTotalOutput, self.currentTotalCostOfProductionOrder, self.currentTotalCostOfUnusedFactors,self.inventories,\
               self.inProgressInventories, self.currentTotalLostProduction, self.currentTotalCostOfLostProduction)
    
    def makeBalancesheet(self):
        if t()==0: self.initialInventories=0 
        else: self.initialInventories=self.finalInventories
        
        self.totalCosts=self.currentTotalCostOfProductionOrder + self.currentTotalCostOfUnusedFactors
        if params['usingMarkup']: self.finalInventories /= (1+self.markup) 
        self.finalInventories=self.inventories+self.inProgressInventories
        if params['usingMarkup']: self.finalInventories *= (1+self.markup) #to open the possibility of having a floating markup
            
        self.profits=self.finalInventories-self.totalCosts-self.initialInventories
        self.addedValue=self.profits+self.totalCosts
        
        self.myBalancesheet[t(), 0]=self.dimensionalClass
        
        self.myBalancesheet[t(), 1]=self.initialInventories
        self.myBalancesheet[t(), 2]=self.totalCosts
        self.myBalancesheet[t(), 3]=self.finalInventories
        self.myBalancesheet[t(), 4]=self.profits
        
        self.myBalancesheet[t(), 5]=self.addedValue
        self.myBalancesheet[t(), 6]=self.currentTotalOutput
        self.myBalancesheet[t(), 7]=self.currentTotalCostOfProductionOrder
        self.myBalancesheet[t(), 8]=self.currentTotalCostOfUnusedFactors
        self.myBalancesheet[t(), 9]=self.currentTotalLostProduction
        self.myBalancesheet[t(), 10]=self.currentTotalCostOfLostProduction
        self.myBalancesheet[t(), 11]=self.totalCostOfLabor
        self.myBalancesheet[t(), 12]=self.totalCostOfCapital
        
        
    
###########################################################################################################################


class ProductiveProcess():
    def __init__(self, targetProductionOfThePeriod:float, requiredLabor:int, requiredCapital:float, duration:int):
        
        self.targetProductionOfThePeriod=targetProductionOfThePeriod
        self.requiredLabor = requiredLabor
        self.requiredCapital = requiredCapital
        self.duration = duration
        self.productionClock=0
        self.hasResources= False
        
    #def step(self, productionOrder)->tuple:
    def step(self)->tuple:
        
        lostProduction=0
        self.costOfLostProduction=0
        self.productionClock += 1
        self.failure=False
        
        # production failure
        if params['probabilityToFailProductionChoices'] >= rng.random():
            self.failure=True
            #print("failure",flush=True)
            lostProduction=self.targetProductionOfThePeriod*self.productionClock
            self.targetProductionOfThePeriod=0
            self.costOfLostProduction=(params['wage']* self.requiredLabor+\
                                       params['costOfCapital']* self.requiredCapital)*self.productionClock
            self.duration = self.productionClock   

        return(self.targetProductionOfThePeriod, self.requiredLabor, self.requiredCapital, \
               lostProduction, self.costOfLostProduction)

===================================================================================================

## 3

the model

===================================================================================================

In [4]:
class Model:
    
    global params
    PARAMS = params
    
    def __init__(self, params: Dict):
        
        self.totalProduction=[]
        self.totalCostOfProduction=[]
        self.totalCostOfUnusedFactors=[]
        self.totalInventories=[]
        self.totalInProgressInventories=[]
        self.totalLostProduction=[]
        self.totalCostOfLostProduction=[]
        
        #the context and the runner are created in step 1 
      
        runner.schedule_event(          0.0,     self.initGhosts) 
        runner.schedule_repeating_event(0.0,  1, self.counter)
        runner.schedule_repeating_event(0.1,  1, self.firmsProducing)
        runner.schedule_repeating_event(0.2,  1, self.enterprisesMakingBalancesheet) #enterprises=firms+banks
        
        runner.schedule_stop(params['howManyCycles'])
        runner.schedule_end_event(self.finish)
        
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################
        
        #importing csv file containing info about firms 
        #share of firms of that class, L min, L max, K min, K max, duration min, duration max, recipe, 
        #L prod, max order production, assets' useful life, markup, order observation frequency, dimensional class
        with open('firm-features.csv', newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
            next(firmReader)
            
            self.rowNumber=0
            k=0
            #for each record in firm-features.csv
            for row in firmReader:
                print(row)
                for i in range(int(row[0] * params['Firm.count'])// rankNum):
                    labor= rng.integers(row[1], row[2]+1) #+1 because integers exclude extremes
                    capital= row[3] + rng.random()*(row[4] -row[3])
                    duration= rng.integers(row[5], row[6]+1) 
                    recipe= row[7] #K/L 
                    laborProductivity= row[8]
                    maxOrderProduction= row[9]
                    avgAssetsUsefulLife=row[10]  #https://www.oecd.org/sdd/productivity-stats/43734711.pdf
                    markup=row[11]
                    orderObservationFrequency=rng.integers(row[12], row[13]+1)
                    dimensionalClass=self.rowNumber
                    aFirm =Firm(k, rank, labor, capital, duration, recipe, laborProductivity,\
                                maxOrderProduction, avgAssetsUsefulLife, markup, orderObservationFrequency,\
                                dimensionalClass)
                    context.add(aFirm)
                    k += 1 # counting agents of the rows
                self.rowNumber += 1

        
    #initialize ghosts by sending them in the ranks before starting the simulation
    def initGhosts(self):
        pass
    
    #count the cycles number
    def counter(self):
        if int(t()) % params["tickNumber.betweenChecks"] == 0: 
            print("rank", rank, "tick", t(), flush=True)
            
  
    
    def firmsProducing(self):
        self.totalProduction.append([0]*(self.rowNumber)) #for each cycle adds a sub-list of lenght number of dimClass
        self.totalCostOfProduction.append([0]*(self.rowNumber))
        self.totalCostOfUnusedFactors.append([0]*(self.rowNumber))
        self.totalInventories.append([0]*(self.rowNumber))
        self.totalInProgressInventories.append([0]*(self.rowNumber))
        self.totalLostProduction.append([0]*(self.rowNumber))
        self.totalCostOfLostProduction.append([0]*(self.rowNumber))
        
        for aFirm in context.agents(agent_type=0):
            #print(aFirm.dimensionalClass)
            aFirm.receivingNewOrder(rng.random()*aFirm.maxOrderProduction)
            tupleOfProductionResults = aFirm.produce()

            self.totalProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[0]
            self.totalCostOfProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[1]
            self.totalCostOfUnusedFactors[t()][aFirm.dimensionalClass] += tupleOfProductionResults[2]
            self.totalInventories[t()][aFirm.dimensionalClass] += tupleOfProductionResults[3]
            self.totalInProgressInventories[t()][aFirm.dimensionalClass] += tupleOfProductionResults[4] 
            self.totalLostProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[5]
            self.totalCostOfLostProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[6]
            
    def enterprisesMakingBalancesheet(self):
        for aFirm in context.agents(agent_type=0):
            aFirm.makeBalancesheet()
    
    
    
    #finish
    def finish(self):
        print("cpu time - calculating phase", Tc(), "rank", rank, flush=True)
        
        # infos for data_analysis.ipynb
        with open('plotInfo.csv', 'w', newline='')\
          as file:
            writer = csv.writer(file)
            writer.writerow((params["log_file_root"],rankNum,\
                             context.size(agent_type_ids=[0])[0]))
        
        """
        print("\n total production", self.totalProduction, flush=True)
        print("\n total cost of production", self.totalCostOfProduction, flush=True)
        print("\n total cost of unused factors", self.totalCostOfUnusedFactors, flush=True)
        print("\n total inventories", self.totalInventories, flush=True)
        print("\n total in progress inventories", self.totalInProgressInventories, flush=True)
        print("\n total lost production", self.totalLostProduction, flush=True)
        print("\n total cost of lost production", self.totalCostOfLostProduction, flush=True)     
        """
        
        names=["_total_production_","_total_cost_of_production_","_total_cost_of_unused_factors_",
               "_total_inventories_","_total_in_progress_inventories_",
               "_total_lost_production_","_total_cost_of_lost_production_"]
        contents=[self.totalProduction,self.totalCostOfProduction,
                  self.totalCostOfUnusedFactors,
                  self.totalInventories,self.totalInProgressInventories,
                  self.totalLostProduction,self.totalCostOfLostProduction]
        
        #os.makedirs(params["log_file_root"], exist_ok = True)

        
        for s in range(len(names)):
            with open(params["log_file_root"]+names[s]+str(rank)+'.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                for k in range(params["howManyCycles"]):
                    writer.writerow(contents[s][k])
                    
        fileList=[] 
        
        for aFirm in context.agents(agent_type=0):        
            #save balancesheets
            with open(params["log_file_root"]+'_'+str(aFirm.uid)+'.csv', 'w', newline='') as file:
                np.savetxt(file, aFirm.myBalancesheet, delimiter=",")
                #create the list of the files containing the balancesheets
                fileList.append(params["log_file_root"]+'_'+str(aFirm.uid)+'.csv')
        
        #save list of balancesheets file names
        with open(params["log_file_root"]+'_balancesheetList_'+str(rank)+'.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(fileList)
        
        print("cpu time - finishing phase", Tc(), "rank", rank, flush=True)
        print("THE END!", flush=True)
    
    def start(self):
        runner.execute()

=========================================================================================================

## 4

run the model

==========================================================================================================

In [5]:
def run(params: Dict):
    
    model = Model(params) 
    model.start()
    
run(params)

[0.1, 40.0, 80.0, 400.0, 800.0, 5.0, 10.0, 5.0, 0.8, 100.0, 12.0, 0.3, 15.0, 20.0]
[0.3, 20.0, 50.0, 100.0, 400.0, 2.0, 4.0, 2.0, 0.7, 50.0, 12.0, 0.2, 10.0, 15.0]
[0.6, 2.0, 18.0, 5.0, 40.0, 1.0, 1.0, 0.5, 0.6, 10.0, 12.0, 0.1, 5.0, 10.0]
rank 0 tick 0
rank 0 tick 10
rank 0 tick 20
rank 0 tick 30
rank 0 tick 40
rank 0 tick 50
rank 0 tick 60
rank 0 tick 70
rank 0 tick 80
rank 0 tick 90
rank 0 tick 100
cpu time - calculating phase 0.16072499999999978 rank 0
cpu time - finishing phase 0.1871649999999998 rank 0
THE END!
